# Ordinary Least Squares (OLS) regression

### Predictive modelling with machine learning

#### Lecturer: Vegard H. Larsen

### Start by generating some data

In [59]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [60]:
# Set the number of regressors or futures
n_features = 10

# Set the number of observations
n_samples = 50

# Set intercept to True/False
has_intercept = True

# Generate some random futures from a Uniform(0,1)
X = np.random.rand(n_samples, n_features)

# Generate the true coefficients, also from a Uniform(0,1)
coeffs = np.random.rand(n_features).round(2)

# The data generating process is y = intercept + sum_i coeff_i * x_i + 0.2*u_i
# where u_i is normally distributed error term,  N(O,1).
if has_intercept:
    intercept = np.random.rand(1).round(2)[0]
    y = intercept + np.dot(X, coeffs) + 0.2*np.random.randn(n_samples)
    y_true = intercept + np.dot(X, coeffs)
else:
    intercept = 0
    y = np.dot(X, coeffs) + 0.2*np.random.randn(n_samples)
    y_true = np.dot(X, coeffs)

In [61]:
print(f'The true coefficients are/is: {coeffs}')
if has_intercept:
    print(f'The true intercept is: {intercept}')

The true coefficients are/is: [0.01 0.58 0.6  1.   0.72 0.82 0.42 0.71 0.23 0.47]
The true intercept is: 0.95


In [62]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming n_features, X, y, y_true, intercept, coeffs are already defined
if n_features == 1:
    plt.scatter(X, y, label='Data')
    plt.plot(X, y_true, color='red', label=f'DGP ($y={intercept} + {coeffs[0]}x$)')

    # Randomly select 20 points
    indices = np.random.choice(range(len(X)), 20, replace=False)
    for i in indices:
        # Draw line from point to regression line
        plt.plot([X[i], X[i]], [y[i], y_true[i]], color='green', linestyle='--')

    plt.legend()
    plt.xlabel('$x$-value')
    plt.ylabel('$y$-value')
    plt.show()
else:
    print('Can only plot a two-dimensional figure.')


Can only plot a two-dimensional figure.


The goal now is to recover the data generating process (DGP) by using the observed data, $X$.

### Basic OLS recap:

The ordinary least squares (OLS) method is a linear regression technique that finds the coefficients (also called weights or parameters) that minimize the sum of the squared residuals between the predicted values and the true values. Mathematically, this can be written as the following optimization problem

$$\arg\min_{\beta} \sum_{i=1}^n (y_i - \beta^T x_i)^2$$

where $x_i$ is the $i$-th feature vector, $y_i$ is the $i$-th target, $\beta$ is the coefficient vector (also called the betas). The optimal values of $\beta$ are the OLS coefficients.

To solve this optimization problem, we can set the derivative of the objective function with respect to $\beta$ to zero and solve for $\beta$. This leads to the following closed-form solution for the OLS coefficients:

$$\beta = (X^T X)^{-1} X^T y$$

where $X$ is the feature matrix with shape (n_samples, n_features), $y$ is the target vector with shape (n_samples).

We can easily solve for $\beta$ using NumPy:

In [63]:
if has_intercept == True:
    X_ = np.column_stack((np.ones(n_samples), X))    
    betas = np.linalg.inv(X_.T@X_)@(X_.T@y)
    print(f'The estimated coefficients are: { betas[1:].round(3)}')
    print(f'The estimated intercept is: { betas[0].round(3)}')
else:
    betas = np.linalg.inv(X.T@X)@(X.T@y)
    print(f'The estimated coefficients are: { betas.round(3)}')

The estimated coefficients are: [0.042 0.646 0.75  0.791 0.694 0.878 0.509 0.877 0.174 0.682]
The estimated intercept is: 0.749


### Let's write a few more lines of code using OOP

We will now use object-oriented programming (OOP) when coding up the ordinary least squares (OLS) method. This is the way every serious package in Python is written, and it is smart to start to think in this framework right away.

In [64]:
class OLS:
    def __init__(self, fit_intercept=True):
        self.fit_intercept = fit_intercept
        self.coeffs = None
        self.intercept = None

    def fit(self, X, y):
        # Add intercept term to X if fit_intercept is True
        if self.fit_intercept:
            X = np.hstack([np.ones((X.shape[0], 1)), X])

        # Solve the least squares problem
        XTX = np.dot(X.T, X)
        XTY = np.dot(X.T, y)
        self.coeffs = np.linalg.solve(XTX, XTY)

        # Extract intercept term if fit_intercept is True
        if self.fit_intercept:
            self.intercept = self.coeffs[0]
            self.coeffs = self.coeffs[1:]

    def predict(self, X):
        # Add intercept term to X if fit_intercept is True
        if self.fit_intercept:
            X = np.hstack([np.ones((X.shape[0], 1)), X])
            return np.dot(X, np.hstack([self.intercept, self.coeffs]))
        return np.dot(X, self.coeffs)

In [65]:
# Then we can use the class as follows:

model = OLS(fit_intercept=has_intercept)
model.fit(X, y)
print(f'The estimated coefficients are: {model.coeffs.round(3)}')
print(f'The estimated intercept is: {model.intercept:.3f}')

The estimated coefficients are: [0.042 0.646 0.75  0.791 0.694 0.878 0.509 0.877 0.174 0.682]
The estimated intercept is: 0.749


In [66]:
model.predict(X)

array([4.72220542, 3.2197737 , 3.08473903, 4.40243285, 4.22691842,
       3.23274087, 4.14106368, 3.25210277, 3.15621231, 3.82182331,
       4.42016605, 3.781478  , 2.80927904, 4.05488809, 3.67817714,
       4.68660717, 2.74839961, 4.61020253, 4.61852001, 3.09034525,
       3.93022215, 2.89289161, 4.61740894, 3.99139584, 3.7955042 ,
       4.09593135, 3.04442985, 3.90517668, 4.24501438, 3.93559573,
       4.08266333, 4.26067852, 3.55126591, 3.19958806, 3.73663152,
       3.55815347, 4.61446708, 4.66322182, 3.43611451, 2.79927118,
       4.50738991, 2.72880435, 3.96632771, 3.25355701, 4.29153706,
       3.5529088 , 4.21675306, 4.30332217, 3.10272679, 3.65663955])

In [72]:
# We can also use the scikit-learn package to estimate the coefficients
import sklearn.linear_model as lm

model_sk = lm.LinearRegression(fit_intercept=has_intercept)
model_sk.fit(X, y)
print(f'The estimated coefficients are: {model_sk.coef_.round(3)}')
print(f'The estimated intercept is: {model_sk.intercept_.round(3)}')

The estimated coefficients are: [0.042 0.646 0.75  0.791 0.694 0.878 0.509 0.877 0.174 0.682]
The estimated intercept is: 0.749
